# Chapter 16 - Visual inertial fusion

An IMU, alsocalled **Interial Measurement Unit**, is a combination of six sensors: Three **Gyroscope** measure the angular velocity (angle change) in each direction, while three **accelerometers** measure linear acceleration, also in each direction.

![Interial Measurement Unit](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_16/interial_measurement_unit.png)
*Figure 1: Interial Measurement Unit. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/13_visual_inertial_fusion.pdf)*

Micro Electro Mechanical System (MEMS) IMUs are cheap, power efficient and lightweight and used in all smartphones or drones today. Their optical counterpart however can range in price from 20'000 to 100'000 dollars, mechanical IMUs can even be up to 1'000'000 Dollars. For our applications, only MEMS IMUs are important, we are therefore only covering their mechanics here in this chapter. 

### MEMS Accelerometer

Accelerometer measure their internal acceleration in the direction of their construction. The acceleration is measured by a capacitive divider with a certian mass which is attached at both ends to two springs. When the Mass is accelerated in the direction of the springs, one spring will extend and the other will shrink, displacing the capactive filter and creating a electric signal at one of its ends. The Mass is dumped by gas which is sealed inside the IMU. Note that this device can only measure the acceleration along the springs, not perpendicular to them. We therefore need three perpendicular Accelerometers to measure 3D movement. 

![Accelerometer](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_16/accelorometer.png)
*Figure 2: Accelerometer. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/13_visual_inertial_fusion.pdf)*

The measured data we get out of an accelorometer is the real acceleration minus G-Force, multiplied by the Rotation from the previous frame to the current frame + IMU bias + noise. 

![Measurement of Acceleration](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_16/acceleration_measurement.png)
*Figure 3: Measurement of Acceleration. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/13_visual_inertial_fusion.pdf)*

The formula states that the *measured* acceleration from frame B(ody) with respect to frame W(orld), measured in frame B, equals the true acceleration from B to W, **but** measured in frame W, minus the G-Force measured in frame W. To put this in frame B, we have to apply the rotation from frame W to frame B. To all this, we of course have to add a bias and noise.


## Camera-IMU System

IMUs can help VO in situations with low texture, high dynamic range and high-speed motion since they rely on intrinsic data and not extrinsic vision information. However, using an IMU without vision will often not work since pure iMU integration will drift drastically over time. IMUs measure acceleration, so we need to integrate once to get the speed, and integrate again to get the position, meaning that the error is proportional to t<sup>2</sup>. So after 1 minute of measuring, cheap IMUs will have a positional error of over 2.2 killometers. 

However, the combination of IMUs and Camera VO is perfect since their specifications are complementary. Cameras are precise in slow motion and provide rich information about the outer world, while having a limited output rate, being scale ambiguous and lack robustness in many scenarios. IMUs have their own disadvantage, like drift, large uncertainty for low acceleration/angular velocity, and gravity/acceleration ambiguity. But that's no problem, VO is perfect in low acceleration/rotation scenarios and gravity has no influence. However, the advantages of IMUs being robust, having high throughput, being extremely accurate in high acceleration scenarios and being able to predict the future position are all characteristics where the IMU can extend the VOs weaknesses. 

In order to use the IMU effectively, it has to be in the same coordinate system as our camera. We therefore need to transform the IMU world measurement W into the body frame B such that the calculated IMU position can be used as an estimate for the camera position as well. Then, we transform all cameras into the body frame as well. We can therefore use multiple cameras with a single IMU to get the cameras new position & rotation estimates. 

### MEMS Gyroscopes

Gyroscopes measure the rotation of the sensor in the direction of their construction by having a vibrating structure. When a rotational force is applied, the Coriolis force will move the vibrating mass perpendicular to its vibration. The distance by which the mass is moved corresponds to the speed of angular rotation. 

In the image below, the gyroscope vibrates along the x axis and a rotation is applied on the x axis. The coliolis force is perpendicular to both, so on the y axis, while the sign of the force indicates positive or negative rotation, and the magnitude indicates the rotational force. Note that we again need three perpendicular gyroscopes to measure rotation in all three axis. 

![Gyroscope](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_16/gyroscope.png)
*Figure 3: Gyroscope. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/13_visual_inertial_fusion.pdf)*

The measured angular velocity from the IMU at a time t is equal to the real angular velocity + IMU bias + noise. 

![Measurement of Angular Velocity](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_16/angular_velocity_measurement.png)
*Figure 4: Measurement of Angular Velocity. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/13_visual_inertial_fusion.pdf)*

The formulate states that the *measured* angular velocity from frame B with respect to frame W, measured in frame B, equals the true angular velocity from frame W to frame B, measured in frame B. To all this, we of course have to add a bias and noise.

### Noise and Bias

We have seen that both IMU parts introduce noise and bias. The noise is just additive gaussian noise for both sensors. Looking at the bias, we observe that the derivative of the bias will again be white gaussian noise, which is good since it makes the noise estimatable. However, the exact bias can change every time the IMU is started due to temperature and pressure. 

### Poition estimate
To estimate the posotion **p<sub>wt2</sub>** at timestamp 2 in the World W using an IMU only, we need the initial position from the previous frame at time t1, **p<sub>wt1</sub>**. To this, we add the velocity **v<sub>Wt1</sub>** at frame 1 * the time passed, **(t<sub>2</sub> - t<sub>1</sub>)**. The velocity in the previous frame has to be known sice our IMU only measures *change* in velocity. To this model, we can now add the sensory data from the IMU. We take the double integral (acceleration -> speed -> position) from the Rotation **R<sub>Wt</sub>** (given by the gyroscope), multiply it with the measured acceleration, from which we substract our bias estimate, and add the g-force. The g-force is added since in the measurement of the IMU itself, the g-force was already subtracted, so to zero it out we have to add it now.

![IMU Model integration](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_16/imu_mode_integration.png)
*Figure 5: IMU Model integration. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/13_visual_inertial_fusion.pdf)*
